In [35]:
#Importing the libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [36]:
df=pd.read_csv("irrigation_machine.csv").iloc[:,1:]
df.head()

,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,parcel_0,parcel_1,parcel_2
0,1.0,2.0,1.0,7.0,0.0,1.0,1.0,4.0,0.0,3.0,...,8.0,1.0,0.0,2.0,1.0,9.0,2.0,0,1,0
1,5.0,1.0,3.0,5.0,2.0,2.0,1.0,2.0,3.0,1.0,...,4.0,5.0,5.0,2.0,2.0,2.0,7.0,0,0,0
2,3.0,1.0,4.0,3.0,4.0,0.0,1.0,6.0,0.0,2.0,...,3.0,3.0,1.0,0.0,3.0,1.0,0.0,1,1,0
3,2.0,2.0,4.0,3.0,5.0,0.0,3.0,2.0,2.0,5.0,...,4.0,1.0,1.0,4.0,1.0,3.0,2.0,0,0,0
4,4.0,3.0,3.0,2.0,5.0,1.0,3.0,1.0,1.0,2.0,...,1.0,3.0,2.0,2.0,1.0,1.0,0.0,1,1,0


In [37]:
X=df.iloc[:,:20]
y=df.iloc[:,-3:]

In [38]:
#Splitting the data in train and test

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=42)

In [39]:
#Standardization the data

from sklearn.preprocessing import StandardScaler

scaler_x=StandardScaler()

scaler_x.fit(X_train)

X_train=pd.DataFrame(scaler_x.transform(X_train),columns=X_train.columns)

X_test=pd.DataFrame(scaler_x.transform(X_test),columns=X_test.columns)

In [40]:
#Creating the model

early=EarlyStopping(monitor="val_accuracy", patience=3)

model=Sequential()

model.add(Dense(256,activation="relu",input_dim=X_train.shape[1]))

model.add(BatchNormalization())

model.add(Dense(128,activation="relu"))

model.add(BatchNormalization())

model.add(Dense(32,activation="relu"))

model.add(BatchNormalization())

model.add(Dense(3,activation="sigmoid"))

model.compile(optimizer="adam",metrics="accuracy",loss="binary_crossentropy")

history=model.fit(X_train,y_train,epochs=200,batch_size=32,validation_split=0.2,callbacks=[early])


Epoch 1/200
35/35 [==============================] - 3s 18ms/step - loss: 0.5799 - accuracy: 0.4580 - val_loss: 0.5610 - val_accuracy: 0.5643
Epoch 2/200
35/35 [==============================] - 0s 7ms/step - loss: 0.3711 - accuracy: 0.5232 - val_loss: 0.4857 - val_accuracy: 0.4571
Epoch 3/200
35/35 [==============================] - 0s 7ms/step - loss: 0.3029 - accuracy: 0.5366 - val_loss: 0.4200 - val_accuracy: 0.4607
Epoch 4/200
35/35 [==============================] - 0s 6ms/step - loss: 0.2616 - accuracy: 0.5562 - val_loss: 0.3714 - val_accuracy: 0.4571


In [41]:
model.evaluate(X_test,y_test)

19/19 [==============================] - 0s 3ms/step - loss: 0.3775 - accuracy: 0.4883


[0.3775310218334198, 0.4883333444595337]

In [42]:
##Tuning hyperparameters

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score,RandomizedSearchCV

def model_function(optimizer="adam",activation="relu"):
    model=Sequential()
    model.add(Dense(100,activation=activation,input_dim=X_train.shape[1]))
    model.add(BatchNormalization())
    model.add(Dense(50,activation=activation))
    model.add(BatchNormalization())
    model.add(Dense(3,activation="sigmoid"))
    model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics="accuracy")
    return model

modelo=KerasClassifier(build_fn=model_function,epochs=3,batch_size=16,verbose=0)

parameters={
    "optimizer":["adam","sgd","rmsprop"],
    "activation":["relu","sigmoid"],
    "epochs":[10,20,40],
    "batch_size":[16,32,64]
}

random=RandomizedSearchCV(modelo,param_distributions=parameters,cv=3)

results=random.fit(X_train,y_train)

print("Best Score", results.best_score_)
print("="*30)
print("Best Parameters" , results.best_params_)

Best Score 0.6357108354568481
Best Parameters {'optimizer': 'sgd', 'epochs': 40, 'batch_size': 32, 'activation': 'sigmoid'}


In [47]:
##Checking the performace
cross_val_score(results,X_test,y_test,cv=3).mean()

0.5716666579246521

In [98]:
##Tuning layers and neurons

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score,RandomizedSearchCV

def model_function_2(dense_layers_extra=2,nn=100):
    model=Sequential()
    model.add(Dense(units=nn,activation="relu",input_dim=X_train.shape[1]))
    
    for i in range(1,dense_layers_extra+1):
        model.add(BatchNormalization())
        model.add(Dense(units=nn, activation="relu"))
        
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")
    
    model.add(Dense(3,activation="sigmoid"))
    
    return model

modelo=KerasClassifier(build_fn=model_function_2,epochs=3,batch_size=16,verbose=0)

parameters={
    "dense_layers_extra":[1,2,3,4],
    "nn":[50,100,200,400]
}

random=RandomizedSearchCV(modelo,param_distributions=parameters,cv=3)

results=random.fit(X_train,y_train)

print("Best Score", results.best_score_)
print("="*30)
print("Best Parameters" , results.best_params_)


Best Score 0.6535981297492981
Best Parameters {'nn': 200, 'dense_layers_extra': 2}


In [96]:
model_function_2(dense_layers_extra=0,nn=100).summary()

Model: "sequential_433"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1279 (Dense)          (None, 100)               2100      
                                                                 
 dense_1280 (Dense)          (None, 3)                 303       
                                                                 
Total params: 2,403
Trainable params: 2,403
Non-trainable params: 0
_________________________________________________________________


In [99]:
##Checking the performace
cross_val_score(results,X_test,y_test,cv=3).mean()

0.6066666642824808